In [149]:
import pandas as pd
import numpy as np
import os
import pprint
import binascii
import mnemonic
import bip32utils
from py_crypto_hd_wallet import HdWallet, HdWalletSaver, HdWalletFactory, HdWalletCoins, HdWalletSpecs, HdWalletWordsNum
from bipwallet import wallet
from py_hd_wallet import wallet
import ast
import subprocess
import json
from pathlib import Path
import subprocess
import requests
from dotenv import load_dotenv

from constants import *
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from web3 import Web3
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from eth_account import Account
load_dotenv()
# df.iloc[0, df.columns.get_loc('symbol')]  getting value of a column from the value of the index

True

In [150]:
# connect to local ETH using geth
w3 = Web3(Web3.HTTPProvider("http://127:0:0:1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

## Setup constants

In [49]:
BTC = 'btc'
ETH = 'eth'
BTCTEST = 'btc-test'

## Generate a Mnemonic

In [31]:
#MNEMONIC = "picnic later boss nurse main crater renew tongue orient echo path buffalo"
#MNEMONIC = "horror true clip rail direct siege robot kid benefit love quarter topic"
NMEMONIC = "become wool prosper pave heavy eagle helmet discover duck cattle nurse deliver"

In [37]:
mnemonic = os.getenv('MNEMONIC')

## Deriving Wallet Keys

In [ ]:
# Define function to derive wallet
def derive_wallets(mnemonic, coin, numderive):
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    return json.loads(output)

In [87]:
# wallet keys
coins = {'btc-test': derive_wallets(MNEMONIC, 'btc-test', 3), 'eth':derive_wallets(MNEMONIC, 'eth', 3)}
print(json.dumps(coins, indent=5, sort_keys = True))

{
     "btc-test": [
          {
               "address": "mg64PnnLd8e6cXe2MTHPZe5YZRqL7JmoMM",
               "index": 0,
               "path": "m/44'/1'/0'/0/0",
               "privkey": "cQC5EEi6b3Sd23va3hsm7ELd3an87DBcamf7WARCrjkEgGLYxsHQ",
               "pubkey": "031396cd4fec4d091a1d04ce4dec305e445a4835b396d2f965d0a0a614c1ba29ac",
               "pubkeyhash": "0642d15e302a6bcc827fe7dbbcaedcf34b00528d",
               "xprv": "tprv8jvwznHSrioko5FyuWBukmJoPbvMZcEhopj8JngNcv4WTBewfWSEy7Lfe7jUZeawkarE818DZWQrASVZ7MBPQzpCgNZqrnPLy9oCvjAN9Ek",
               "xpub": "tpubDGcz9CKh16VRgYHmo9rWAAxuxdSHiwRcP8KubJig3BruHfuiHuFq9bxXpGBSxK8EvvaN8sKizD8d9XAVs1xGPXr7bPnZzCNqv9nv6TmJbn6"
          },
          {
               "address": "mwzePbUy2sZ6uejXaEwcXd6N8yEybVxw7f",
               "index": 1,
               "path": "m/44'/1'/0'/0/1",
               "privkey": "cRppt1GT1EhhH5dGk62GgctPuXC7UqXZjmYJHuXKTFa7Txm3Bu6E",
               "pubkey": "03ba70eb9594dd18c07d6b4706a5f68daf222815bf1

In [92]:
btc_key = coins['btc-test'][0]['privkey']
eth_key = coins['eth'][0]['privkey']
eth_key

'0xe8d29a3c15b28a286f16db2b3f33bbb2986037a9a3fdc5019f86a974b1b8ded1'

# Linking The Transaction Signing Libraries

In [143]:
# Creates an account from a private key
def priv_key_to_account(coin, priv_key):
    global account
    if coin == "eth":
        account = Account.privateKeyToAccount(priv_key)
    elif coin == "btc":
        account = PrivateKeyTestnet(priv_key)
        return account
    else:
        return print("Please use lower case. If this error persists, please use another coin.")
    

In [109]:
ETH_account = priv_key_to_account(ETH, eth_key)
ETH_account

In [110]:
BTC_account = priv_key_to_account(BTC, btc_key)
BTC_account

<PrivateKeyTestnet: mg64PnnLd8e6cXe2MTHPZe5YZRqL7JmoMM>

In [141]:
# This will create the transaction raw and unsigned. Serves as a preview transaction before sending it.
def create_tx(coin, account, recipient, amount):
    global tx_data
    if coin =='eth':
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        tx_data = {
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
        return tx_data
    elif coin == 'btc':
        tx_data = PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])
        return tx_data
    else:
        return print("Please use lower case. If this error persists, please use another coin.")

In [142]:
# Signs the transaction created and sends it to a designated network
def send_tx(coin, account, recipient, amount):
    
    if coin == 'eth':
        tx = create_tx(coin,account, recipient, amount)
        signed_tx = account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == 'btc':
        #tx_send = create_tx(coin, account, recipient, amount)
        signed = account.sign_transaction(tx_data)
        NetworkAPI.broadcast_tx_testnet(signed)
        return signed

# Send Some Transactions

#### Bitcoin transaction

In [138]:
# Create Tx
BTC_recipient = 'mwzePbUy2sZ6uejXaEwcXd6N8yEybVxw7f'
Bitcoin_amount = 0.0001

my_transaction = create_tx(BTC, BTC_account, BTC_recipient, Bitcoin_amount)
json.loads(my_transaction)

{'unspents': [{'amount': 100000,
   'confirmations': -1805198,
   'script': '76a9140642d15e302a6bcc827fe7dbbcaedcf34b00528d88ac',
   'txid': '6fb59dbbd856ab12e2f40d71a0975af27f053df37dc014e7437d7d0c86039f62',
   'txindex': 1,
   'type': 'p2pkh',
   'vsize': 148,
   'segwit': False}],
 'outputs': [['mwzePbUy2sZ6uejXaEwcXd6N8yEybVxw7f', 10000],
  ['mg64PnnLd8e6cXe2MTHPZe5YZRqL7JmoMM', 41184]]}

In [147]:
# send transaction
sent_btc = send_tx(BTC, BTC_account, BTC_recipient, Bitcoin_amount)
print(sent_btc)

0100000001629f03860c7d7d43e714c07df33d057ff25a97a0710df4e212ab56d8bb9db56f010000006b483045022100e7a444c25a116bcacd21fb5763598f3c6993d8b9c1d85bdd125d532e3b310e3902204955f8c6efc1ae1c7c6dc3f8f9630b9463b3af711bf08d444cb5e3388d599a700121031396cd4fec4d091a1d04ce4dec305e445a4835b396d2f965d0a0a614c1ba29acffffffff0210270000000000001976a914b4beb6ad2cd967c6de8924f1a51f68599520471a88ace0a00000000000001976a9140642d15e302a6bcc827fe7dbbcaedcf34b00528d88ac00000000


#### Ethereum transaction Local PoA

In [158]:
# connecting to HTTP with private key
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0xe8d29a3c15b28a286f16db2b3f33bbb2986037a9a3fdc5019f86a974b1b8ded1"))

In [170]:
w3.isConnected()

True

In [164]:
# checking balance with address_1
w3.eth.getBalance("0x272B4096637f964BF9F6DdbB3Dd5457E679B23B2")

30000000000000000000

In [167]:
# create tx
ETH_recipient_to = '0x41179f51e13fb430d3419AA765e6C48cE49b11c9'
Ethereum_amount = 10000000000000000000
my_eth_transaction = create_tx(ETH, ETH_account, ETH_recipient_to, Ethereum_amount)
my_eth_transaction

{'from': '0x272B4096637f964BF9F6DdbB3Dd5457E679B23B2',
 'to': '0x41179f51e13fb430d3419AA765e6C48cE49b11c9',
 'value': 10000000000000000000,
 'gasPrice': 20000000000,
 'gas': 21000,
 'nonce': 0}

In [168]:
# send tx
sent_eth = send_tx(ETH, ETH_account, ETH_recipient_to, Ethereum_amount)
sent_eth

0xf7df706e2cfc28fa5fd05d3a9ace383beb2e6f6e9c00fb88d1c49e634c6ea062


'0xf7df706e2cfc28fa5fd05d3a9ace383beb2e6f6e9c00fb88d1c49e634c6ea062'

In [171]:
# checking balance on recipient account (account that received funds)
w3.eth.getBalance(ETH_recipient_to)

10000000000000000000

In [177]:
# getting latest block to preview successful transaction.
first_transaction = w3.eth.getBlock('latest')
first_transaction

AttributeDict({'number': 33,
 'hash': HexBytes('0x092b209cdb3bf90a006e1c07acb1e9a6e2de86504f5ab464017eeff5fa719aaa'),
 'parentHash': HexBytes('0x051224df8e9f0dbfa9f27336c30a47944e746bda52cc4db1bea0140f1d0b28c7'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRo

**In conclusion: the receiving account (address_2) received their funds of 10000000000000000000 ETH successfully from account 1. Account one now has a little less that 20000000000000000000 ETH. This was possible using gnache nodes connected with my mnemonic on my local machine. The above is the block where the transaction was recorded. Note that the block is placed in a tuple because it can not be updated or altered.**